In [5]:
!pip3 install nltk

zsh:1: /Users/adi/Desktop/TA_Adi/venv/bin/pip3: bad interpreter: /Users/adi/Desktop/geographical_news_scrapper/venv/bin/python: no such file or directory


In [2]:
# Import library-library
import os
import glob

# Data Preparation and Preprocessing
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
NLTK_StopWords = stopwords.words('indonesian')
NLTK_StopWords = set(NLTK_StopWords)

# Word Embedding
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

from sklearn.metrics.pairwise import cosine_similarity

# Input and Expansion Query
from textblob import TextBlob
from nltk.tokenize import wordpunct_tokenize
#from googletrans import Translator

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/adi/nltk_data'
    - '/Users/adi/Desktop/TA_Adi/venv/nltk_data'
    - '/Users/adi/Desktop/TA_Adi/venv/share/nltk_data'
    - '/Users/adi/Desktop/TA_Adi/venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [2]:
def preprocessing(berita):
    s = berita.lower()
    s = s.replace('\n', ' ')
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    tokens = [token for token in s.split(" ") if token != ""]
    T = [t for t in tokens if t not in NLTK_StopWords]
    return T

In [3]:
text=preprocessing('saya mencoba menggunakan preprocessing')
print (text)

['mencoba', 'preprocessing']


In [15]:
df =pd.read_csv("scrapped_news.csv")
df.head()

,title,date,description,source
0,"Usai Kampung Bahari, Kini Kampung Boncos 'Diob...","Kamis, 10 Mar 2022 17:43 WIB\n\n",Penindakan terhadap peredaran\n\n Kapolsek Pal...,www.detik.com
1,5 Fakta Razia Kampung Bahari Berujung Terkuak ...,"Kamis, 10 Mar 2022 08:04 WIB\n\n",", Tanjung Priok, Jakarta Utara, kembali digere...",www.detik.com
2,Bandar Narkoba di Kampung Bahari Sembunyikan S...,"Rabu, 09 Mar 2022 20:59 WIB\n\n",Polisi menyita 22 unit motor dalam penggerebek...,www.detik.com
3,Kampung Narkoba di Deli Serdang Sumut Digerebe...,"Kamis, 10 Mar 2022 04:30 WIB\n\n",Polisi menggerebek\n\n Penggerebekan itu dilak...,www.detik.com
4,"Kode Bandar Narkoba Kampung Bahari, Nyalakan P...","Rabu, 09 Mar 2022 16:50 WIB\n\n","Polisi mengungkapkan\n\n ""Ada petasan sebagai ...",www.detik.com


In [21]:
column = ['Title', 'Date', 'Description',]
df_total = pd.DataFrame()
document_text = []
df =pd.read_csv("scrapped_news.csv")
isi=df['description']
df = df.dropna(subset=['description'], axis=0)
for x in range(0, df.shape[0]):
    text = preprocessing (df.iloc[x, -1])
    document_text.append(df.iloc[x, -1])
    df.iloc[x, -1] = (' '.join(text)) 

In [22]:
teks = []

for i in range(df.shape[0]):
        teks.append(df.iloc[i, -1])

In [23]:
print (teks)

['www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik com', 'www detik co

In [24]:
print(df.info())
print ('-'*90)
print("Panjang Dokumen Teks : ", len(document_text))
#print(len(document_text))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1804 entries, 0 to 1803
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1804 non-null   object
 1   date         1804 non-null   object
 2   description  1804 non-null   object
 3   source       1804 non-null   object
dtypes: object(4)
memory usage: 70.5+ KB
None
------------------------------------------------------------------------------------------
Panjang Dokumen Teks :  1804


In [25]:
df.head()

,title,date,description,source
0,"Usai Kampung Bahari, Kini Kampung Boncos 'Diob...","Kamis, 10 Mar 2022 17:43 WIB\n\n",Penindakan terhadap peredaran\n\n Kapolsek Pal...,www detik com
1,5 Fakta Razia Kampung Bahari Berujung Terkuak ...,"Kamis, 10 Mar 2022 08:04 WIB\n\n",", Tanjung Priok, Jakarta Utara, kembali digere...",www detik com
2,Bandar Narkoba di Kampung Bahari Sembunyikan S...,"Rabu, 09 Mar 2022 20:59 WIB\n\n",Polisi menyita 22 unit motor dalam penggerebek...,www detik com
3,Kampung Narkoba di Deli Serdang Sumut Digerebe...,"Kamis, 10 Mar 2022 04:30 WIB\n\n",Polisi menggerebek\n\n Penggerebekan itu dilak...,www detik com
4,"Kode Bandar Narkoba Kampung Bahari, Nyalakan P...","Rabu, 09 Mar 2022 16:50 WIB\n\n","Polisi mengungkapkan\n\n ""Ada petasan sebagai ...",www detik com


In [27]:
# Simpan teks asli dokumen
joblib.dump(document_text, "document_text.pkl")
# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
df.to_csv('df_total.csv',index=False )


In [28]:
# Fit transform menggunakan TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(teks)

print('tfidf_matrix')
print(tfidf_matrix.shape)

tfidf_matrix
(1804, 3)


In [29]:
joblib.dump(tfidf_vectorizer, "vectorizer.pkl")
joblib.dump(tfidf_matrix, "tfidf.pkl")

['tfidf.pkl']

In [30]:
df_total = pd.read_csv('df_total.csv')
df_total = df_total[pd.notnull(df_total['description'])]
print(df_total.info())
print ('-'*90)
document_text= joblib.load('document_text.pkl')
print(len(document_text))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1804 entries, 0 to 1803
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1804 non-null   object
 1   date         1804 non-null   object
 2   description  1804 non-null   object
 3   source       1804 non-null   object
dtypes: object(4)
memory usage: 70.5+ KB
None
------------------------------------------------------------------------------------------
1804


In [33]:
kueri='penembakan'
kueri=preprocessing(kueri)
kueri= [" ".join (kueri)]
print (kueri)

tfidf_matrix =joblib.load( "tfidf.pkl" )
tfidf_vectorizer = joblib.load( "vectorizer.pkl" ) 
query_vec= tfidf_vectorizer.transform(kueri)
results=cosine_similarity(tfidf_matrix, query_vec).reshape((-1))
#print (results)
print("\n======================\n")
print("Top 10 most similar documents in corpus:")

j = 1

for i in results.argsort()[-10:][::-1]:
    print(j)
    print("No ID Dokumen  : ", i)
    print("Tanggal        : ", df_total.iloc[i,1])
    print("Isi berita     : ", df_total.iloc[i,2])
    print("(Score: %.4f) " % results[i])
    j += 1

['penembakan']


Top 10 most similar documents in corpus:
1
No ID Dokumen  :  1803
Tanggal        :  Senin, 01 Nov 2021 16:29 WIB


Isi berita     :  Subdit Resmob Ditreskrimum Polda Metro Jaya menangkap

 Peristiwa itu terjadi pada Senin (11/10) sekitar pukul 02.30 WIB. Saat itu
korban dan pacarnya tengah menunggu ojek

 "Kemudian muncul empat pelaku dengan dua motor. Saat itu salah seorang pelaku
turun inisial T yang berstatus DPO mengeluarkan satu kalimat 'kamu sudah

 Pacar korban kemudian kabur menyelamatkan diri dan meninggalkan korban di
lokasi. Saat itu pelaku T lalu merampas

 Yusri memastikan aksi tuduhan pelaku itu hanya modus semata. Korban dan pelaku
sebelumnya tidak saling kenal.

 "Jadi memang ini salah satu teknisnya pelaku ke korban untuk merebut barang
korban. Jadi kalimat itu adalah kalimat yang mengaburkan saja," terang Yusri.

 Tiga orang pelaku kini telah berhasil ditangkap polisi. Ketiga pelaku
berinisial RP, MG, dan MR.

 Namun, satu pelaku utama dari kasus ters